In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import v2
import numpy as np

In [10]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 10
batch_size = 4
learning_rate = 0.001

transform = v2.Compose(
    [v2.ToTensor(),
     v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

tarin_loder = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


/Users/omilod/Desktop/projects/GigaTorch/env/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified


In [11]:
class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

model = ConvNet()
model.to(device)
critertion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(tarin_loder)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(tarin_loder):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = critertion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print(f'Epoche [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print("Finished training")


with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy: {acc} %')


Epoche [1/10], Step [2000/12500], Loss: 2.2467
Epoche [1/10], Step [4000/12500], Loss: 2.2539
Epoche [1/10], Step [6000/12500], Loss: 2.2477
Epoche [1/10], Step [8000/12500], Loss: 2.4960
Epoche [1/10], Step [10000/12500], Loss: 2.4319
Epoche [1/10], Step [12000/12500], Loss: 2.3993
Epoche [2/10], Step [2000/12500], Loss: 1.7446
Epoche [2/10], Step [4000/12500], Loss: 1.7958
Epoche [2/10], Step [6000/12500], Loss: 1.9598
Epoche [2/10], Step [8000/12500], Loss: 2.0397
Epoche [2/10], Step [10000/12500], Loss: 1.8597
Epoche [2/10], Step [12000/12500], Loss: 1.1347
Epoche [3/10], Step [2000/12500], Loss: 1.2750
Epoche [3/10], Step [4000/12500], Loss: 1.4700
Epoche [3/10], Step [6000/12500], Loss: 0.8299
Epoche [3/10], Step [8000/12500], Loss: 2.4020
Epoche [3/10], Step [10000/12500], Loss: 0.8445
Epoche [3/10], Step [12000/12500], Loss: 1.2414
Epoche [4/10], Step [2000/12500], Loss: 1.1043
Epoche [4/10], Step [4000/12500], Loss: 1.6044
Epoche [4/10], Step [6000/12500], Loss: 1.8810
Epoche 